In [1]:
import tensorflow as tf 
import numpy as np 
from tensorflow.examples.tutorials.mnist import input_data 

In [2]:
#1、加载数据
mnist = input_data.read_data_sets('../../data',one_hot=True)

Extracting ../../data\train-images-idx3-ubyte.gz
Extracting ../../data\train-labels-idx1-ubyte.gz
Extracting ../../data\t10k-images-idx3-ubyte.gz
Extracting ../../data\t10k-labels-idx1-ubyte.gz


In [10]:
#2、构建模型
#(1)定义数据并预处理数据
trX,trY,teX,teY = mnist.train.images,mnist.train.labels,mnist.test.images,mnist.test.labels

trX = trX.reshape(-1,28,28,1)
teX = teX.reshape(-1,28,28,1)

X = tf.placeholder(tf.float32,[None,28,28,1])
Y = tf.placeholder(tf.float32,[None,10])

#(2)初始化权重与定义网络结构
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape,stddev=0.01))
#权重初始化，卷积核的大小为3*3
w = init_weights([3,3,1,32])     #patch为3*3，输入维度为1，输出维度为32
w2 = init_weights([3,3,32,64])   #patch为3*3，输入维度为32，输出维度为64
w3 = init_weights([3,3,64,128])  #patch为3*3，输入维度为63，输出维度为128
w4 = init_weights([128*4*4,625]) #全连接层，输入维度为128*4*4，是上一层的输出数据由三维的转变成一维，输出维度为625
w_0 = init_weights([625,10])     #输出层，输入维度为625，输出维度为10，代表10类(labels)

#定义一个模型函数
#神经网络模型的构建函数，传入以下参数
#X：输入数据
#w：每一层的权重
#p_keep_conv,p_keep_hidden,dropout 要保留神经元比例

def model(X,w,w2,w3,w4,w_0,p_keep_conv,p_keep_hidden):
    #第一组卷积层及池化层，最后dropout一些神经元
    l1a = tf.nn.relu(tf.nn.conv2d(X,w,strides=[1,1,1,1],padding='SAME'))  #l1a为：shape =(?,28,28,32)
    l1 = tf.nn.max_pool(l1a,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME') #l1为：shape=(?,14,14,32)
    l1 = tf.nn.dropout(l1,p_keep_conv)
    
    #第二组卷基层及池化层，最后dropout一些神经元
    l2a = tf.nn.relu(tf.nn.conv2d(l1,w2,strides=[1,1,1,1],padding='SAME')) #l2a为：shape=(?,14,14,64)
    l2 = tf.nn.max_pool(l2a,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME') #l2为：shape=(?,7,7,64)
    l2 = tf.nn.dropout(l2,p_keep_conv)
    
    #第三层卷基层及池化层，最后dropout一些神经元
    l3a = tf.nn.relu(tf.nn.conv2d(l2,w3,strides=[1,1,1,1],padding='SAME')) #l3a为：shape=(?,7,7,128)
    l3 = tf.nn.max_pool(l3a,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME') #l3为：shape=(?,4,4,128)
    l3 = tf.reshape(l3,[-1,w4.get_shape().as_list()[0]]) # reshape to (?,2048)
    l3 = tf.nn.dropout(l3,p_keep_conv) 
    
    #第四层，全连接层
    l4 = tf.nn.relu(tf.matmul(l3,w4))
    l4 = tf.nn.dropout(l4,p_keep_hidden)
    
    #输出层
    pyx = tf.matmul(l4,w_0)
    return pyx #返回预测值

p_keep_conv = tf.placeholder('float')
p_keep_hidden = tf.placeholder('float')
py_x = model(X,w,w2,w3,w4,w_0,p_keep_conv,p_keep_hidden) #得到预测值

#定义损失函数 和优化函数
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x,labels=Y))
train_op = tf.train.RMSPropOptimizer(0.001,0.9).minimize(cost)
predict_op = tf.argmax(py_x,1)

In [ ]:
#3、训练模型及评估模型
batch_size = 128 
test_size =256
init = tf.global_variables_initializer() 
with tf.Session() as sess:
    sess.run(init)
    for i in range(1000):
        training_batch = zip(range(0,len(trX),batch_size),
                             range(batch_size,len(trX)+1,batch_size))
        for start,end in training_batch:
            sess.run(train_op,feed_dict={X:trX[start:end],Y:trY[start:end],
                                        p_keep_conv:0.8,p_keep_hidden:0.5})
        
        test_indices = np.arange(len(teX))
        np.random.shuffle(test_indices)
        test_indices =test_indices[0:test_size]
        print(i,np.mean(np.argmax(teY[test_indices],axis=1)==
                       sess.run(predict_op,feed_dict={X:teX[test_indices],
                                                     p_keep_conv:1.0,
                                                     p_keep_hidden:1.0})))


0 0.94921875
1 0.9921875
2 0.984375
3 0.9921875
4 0.99609375
5 0.98046875
6 0.9921875
7 0.99609375
8 0.9921875
9 0.99609375
10 0.99609375
11 0.99609375
12 0.984375
13 0.9921875
14 0.98828125
15 0.9921875
16 0.99609375
17 0.99609375
18 1.0
19 0.984375


In [6]:
trX.shape

(55000, 784)

In [ ]:
trX = trX.reshape(-1,28,28,1)
teX = teX.reshape(-1,28,28,1)